# Dyadic transpose: `A⍉B`

> The development of mathematics toward greater precision has led, as is well known, to the formalization of large tracts of it, so that one can prove any theorem using nothing but a few mechanical rules... One might therefore conjecture that these axioms and rules of inference are sufficient to decide any mathematical question that can at all be formally expressed in these systems. It will be shown below that this is not the case, that on the contrary there are in the two systems mentioned relatively simple problems in the theory of integers that cannot be decided on the basis of the axioms. --_Kurt Gödel_

In [13]:
⎕IO ← 0
]box on -s=max -t=tree -f=on
]rows on
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→─────┐
│Was ON│
└──────┘

Mastery of the dyadic form of _transpose_, `A⍉B` is -- together with _rank_, `⍤` -- is considered the mark of the accomplished array programmer. Let's see if it's as hard to grasp as its reputation suggests. 

Some other resources on dyadic transpose:
* [APLWiki](https://aplwiki.com/wiki/Transpose#Dyadic_usage)
* Dyalog [docs](https://help.dyalog.com/latest/#Language/Primitive%20Functions/Transpose%20Dyadic.htm)
* Webinar: [The Rank Operator and Dyadic Transpose](https://dyalog.tv/Webinar/?v=zBqdeDJPPRc)

Most programmers have probably had at least some exposure to linear algebra, and so have internalised the concept of the transpose of a matrix: the x-axis becomes the y-axis, and vice-versa. We've alreay met the monadic form:

In [14]:
⊢B ← 3 4⍴9 4 2 7 2 5 4 7 8 6 1 2 6 8 2 9
⍉B

┌→──────┐
↓9 4 2 7│
│2 5 4 7│
│8 6 1 2│
└~──────┘

┌→────┐
↓9 2 8│
│4 5 6│
│2 4 1│
│7 7 2│
└~────┘

We can see what was the x-axis is now the y-axis as we said. If we generalise this a bit, transpose is an operation that _reorders_ axes. It just happens to be the case that in the rank-2 case, there is only one other possible ordering.

In the dyadic form of transpose, we are explicit about the new axis order:

In [15]:
1 0⍉B ⍝ Same result as the monadic form: x-is-y

┌→────┐
↓9 2 8│
│4 5 6│
│2 4 1│
│7 7 2│
└~────┘

So far, so obvious. However... once the rank increases, it becomes harder to visualise perhaps what's going on. One tip is to not think of the array itself having its axes pulled and moved, but to instead think of the _observer_ (you, or "the camera") moving around the array. 

Consider rank-3:

In [16]:
m ← 3 3 3⍴1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3

![dtr](dyadic.png)

Imagine if we want to rearrange this array such that the new view is that indicated by the camera symbol, to see the array as three slices each of:

In [17]:
3 3⍴1 2 3

┌→────┐
↓1 2 3│
│1 2 3│
│1 2 3│
└~────┘

We'd need to rotate the scene 90 degrees clockwise around the current axis 1. So axis 1 stays the same, but axes 0 and 2 changes position:

In [18]:
2 1 0⍉m

┌┌→────┐
↓↓1 2 3│
││1 2 3│
││1 2 3│
││ │
││1 2 3│
││1 2 3│
││1 2 3│
││ │
││1 2 3│
││1 2 3│
││1 2 3│
└└~────┘

When rank goes above 3, it becomes tricker to visualise this way, but instead, think about what each resulting major cell should look like -- typically, when you need dyadic transpose, you have an idea what shape each cell should take, and you can usually work backwards from there to deduce the correct axis ordering.

But perhaps the main question remains -- what is this all useful for? A lot, as it turns out. One helpful clue is that transpose, including its dyadic form, is one of the functions allowing for modified assignment. We talked about that in the section on indexing earlier. By assigning to the dyadic transpose of an array we can "fill" it from a data source that has the elements laid out in a different arrangement.

We can use the same example data as above to illustrate this. Let's say that our data is a ravel of recurring 1, 2, 3 to give 27 elements, but we want it organised as our original matrix we used above, with the major cells being `3 3⍴9/1`, `3 3⍴9/2` and `3 3⍴9/3` respectively

In [19]:
⊢data ← ∊9/⊂1 2 3
mat ← 3 3 3⍴0             ⍝ Empty matrix

┌→────────────────────────────────────────────────────┐
│1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3│
└~────────────────────────────────────────────────────┘

We can now assign through the dyadic transpose to fill our empty matrix in a different axis order:

In [20]:
(2 1 0⍉mat) ← 3 3 3⍴data  
mat

┌┌→────┐
↓↓1 1 1│
││1 1 1│
││1 1 1│
││ │
││2 2 2│
││2 2 2│
││2 2 2│
││ │
││3 3 3│
││3 3 3│
││3 3 3│
└└~────┘

Let's look at a higher-rank example. Given a 6×6 matrix, partition it into four non-overlapping partitions of size 3×3. Can we achieve this using dyadic transpose? Here's our matrix:

In [21]:
⊢mat ← 6 6⍴⍳36

┌→────────────────┐
↓ 0 1 2 3 4 5│
│ 6 7 8 9 10 11│
│12 13 14 15 16 17│
│18 19 20 21 22 23│
│24 25 26 27 28 29│
│30 31 32 33 34 35│
└~────────────────┘

Let's reshape this into an array of shape 2 3 2 3 (rank 4), like so:

In [22]:
⊢r4 ← 2 3 2 3⍴mat

┌┌┌→───────┐
↓↓↓ 0 1 2│
│││ 3 4 5│
│││ │
│││ 6 7 8│
│││ 9 10 11│
│││ │
│││12 13 14│
│││15 16 17│
│││ │
│││ │
│││18 19 20│
│││21 22 23│
│││ │
│││24 25 26│
│││27 28 29│
│││ │
│││30 31 32│
│││33 34 35│
└└└~───────┘

The first cell has turned the first three rows of our original matrix into a rank-3 array where the cells are shape 2 3, essentially "folding" each row in half:

In [23]:
0⌷r4

┌┌→───────┐
↓↓ 0 1 2│
││ 3 4 5│
││ │
││ 6 7 8│
││ 9 10 11│
││ │
││12 13 14│
││15 16 17│
└└~───────┘

We can now look to reorder the axes. After a bit of thought we can say that we need the shape to be 2 2 3 3 before we have a go at partitioning. If we think of the 3×3 at the end as given by the task at hand (our partition sizes), we know that we'll ultimately need two "rows" and two "cols" of those. So we currently have a shape of 2 3 2 3, so we flip the middle two axes:

In [24]:
⊢reorder ← 0 2 1 3⍉r4

┌┌┌→───────┐
↓↓↓ 0 1 2│
│││ 6 7 8│
│││12 13 14│
│││ │
│││ 3 4 5│
│││ 9 10 11│
│││15 16 17│
│││ │
│││ │
│││18 19 20│
│││24 25 26│
│││30 31 32│
│││ │
│││21 22 23│
│││27 28 29│
│││33 34 35│
└└└~───────┘

That looks right: each 3×3 cell is correct. We just need to enclose each rank-2 cell:

In [25]:
⊂⍤2⊢reorder

┌→──────────────────────┐
↓ ┌→───────┐ ┌→───────┐ │
│ ↓ 0 1 2│ ↓ 3 4 5│ │
│ │ 6 7 8│ │ 9 10 11│ │
│ │12 13 14│ │15 16 17│ │
│ └~───────┘ └~───────┘ │
│ ┌→───────┐ ┌→───────┐ │
│ ↓18 19 20│ ↓21 22 23│ │
│ │24 25 26│ │27 28 29│ │
│ │30 31 32│ │33 34 35│ │
│ └~───────┘ └~───────┘ │
└∊──────────────────────┘

## Translating RNA into Protein

Here's an example from [Project Rosalind](http://rosalind.info/problems/list-view/) -- a great problem collection in the field of bioinformatics: http://rosalind.info/problems/prot/

The ask is to take an mRNA sequence and encode specific "codons" into the amino acid alphabet. No, I don't know what that means either, but it doesn't matter: we have a sequence of letters, which when grouped into triplets define a point in a 4×4×4 matrix.

The "codon table" is given as:

```
UUU F      CUU L      AUU I      GUU V
UUC F      CUC L      AUC I      GUC V
UUA L      CUA L      AUA I      GUA V
UUG L      CUG L      AUG M      GUG V
UCU S      CCU P      ACU T      GCU A
UCC S      CCC P      ACC T      GCC A
UCA S      CCA P      ACA T      GCA A
UCG S      CCG P      ACG T      GCG A
UAU Y      CAU H      AAU N      GAU D
UAC Y      CAC H      AAC N      GAC D
UAA Stop   CAA Q      AAA K      GAA E
UAG Stop   CAG Q      AAG K      GAG E
UGU C      CGU R      AGU S      GGU G
UGC C      CGC R      AGC S      GGC G
UGA Stop   CGA R      AGA R      GGA G
UGG W      CGG R      AGG R      GGG G
```
and you'd be excused for not seeing the 4×4×4 matrix immediately popping out at you. However, the question links to a Wikipedia page which has the following helpful illustration:

![codons](./IMG/3D_Genetic_Code.jpg)

So let's take the values from the table given and make the above 3D representation from it, and then solve this Rosalind problem. The "Stop" codons is basically just the "EOL" -- we can call that '.' for simplicity. Picking the values and reshaping those to 4×4×4:

In [9]:
4 4 4⍴'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'

┌┌→───┐
↓↓FLIV│
││FLIV│
││LLIV│
││LLMV│
││ │
││SPTA│
││SPTA│
││SPTA│
││SPTA│
││ │
││YHND│
││YHND│
││.QKE│
││.QKE│
││ │
││CRSG│
││CRSG│
││.RRG│
││WRRG│
└└────┘

Now let's figure out the axis order. Looking at the table (and graph), the coordinate order is U, C, A, G. Let's look at the first few entries in the table again:

```
UUU F      CUU L      AUU I      GUU V
UUC F      CUC L      AUC I      GUC V
UUA L      CUA L      AUA I      GUA V
UUG L      CUG L      AUG M      GUG V
```
We can see that when the leading coordinate changes we get `FFLL`, `LLLL`, `IIIM`, `VVVV`, so our current axis ordering isn't correct. Looking at the graph, we can see that this corresponds to placing the "camera" to the left, looking right -- our first axis is probably where it should be. Let's try to flip the other two:

In [10]:
0 2 1⍉4 4 4⍴'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'

┌┌→───┐
↓↓FFLL│
││LLLL│
││IIIM│
││VVVV│
││ │
││SSSS│
││PPPP│
││TTTT│
││AAAA│
││ │
││YY..│
││HHQQ│
││NNKK│
││DDEE│
││ │
││CC.W│
││RRRR│
││SSRR│
││GGGG│
└└────┘

Hmm, the first row of the first cell _is_ correct, but not the rest, but if the first axis is correct we have no options left, right? Let's try a monadic transpose, too:

In [8]:
0 2 1⍉⍉4 4 4⍴'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'

┌┌→───┐
↓↓FFLL│
││SSSS│
││YY..│
││CC.W│
││ │
││LLLL│
││PPPP│
││HHQQ│
││RRRR│
││ │
││IIIM│
││TTTT│
││NNKK│
││SSRR│
││ │
││VVVV│
││AAAA│
││DDEE│
││GGGG│
└└────┘

That looks correct. Now we can solve this problem:

In [11]:
codon←0 2 1⍉⍉4 4 4⍴'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'

In [12]:
]dinput
prot ← {
    enc ← 0 1 2 3['UCAG'⍳⍵]        ⍝ Convert RNA string from UCAG to 0123
    ¯1↓codon[⊂⍤1⊢(3÷⍨≢enc) 3⍴enc]  ⍝ Group into triplets and index into codon table, and drop Stop.
}

We can try this on the example given in the question:

In [14]:
test ← 'AUGGCCAUGGCGCCCAGAACUGAGAUCAAUAGUACCCGUAUUAACGGGUGA'
⊢r ← prot test
assert 'MAMAPRTEINSTRING'≡r

┌→───────────────┐
│MAMAPRTEINSTRING│
└────────────────┘